In [1]:
"""
[ 협업 필터링 추천 ]
- 1명의 유저와 비슷한 취향의 사람들을 찾아 공통적으로 좋아하는 item추천
"""

'\n[ 협업 필터링 추천 ]\n- 1명의 유저와 비슷한 취향의 사람들을 찾아 공통적으로 좋아하는 item추천\n'

In [2]:
# Data 읽어오기
from google.colab import files
users=files.upload()

Saving user.txt to user.txt


In [3]:
# User Data
import io
import pandas as pd
u_cols=['user_id','age','sex','occupation','zip_code']
users=pd.read_csv('user.txt',sep='|',names=u_cols)
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
# u.item file을 Dataframe으로 읽기 (영화 정보 )
items=files.upload()

Saving u.item to u.item


In [5]:
cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventure','Animation','children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romancce','Sci-Fi','Thriller','War','Western']
movies=pd.read_csv('u.item',sep='|',names=cols,encoding='latin=1')
#movies=movies.set_index('movie_id')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romancce,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [6]:
# u.data file을 DataFrame으로 읽기
data=files.upload()

Saving u.data to u.data


In [7]:
r_cols=['user_id','movie_id','rating','timestamp']
ratings=pd.read_csv('u.data',sep='\t',names=r_cols,encoding='latin=1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
# timestamp 제거
ratings=ratings.drop('timestamp',axis=1)

In [9]:
# movie id, title 빼고 다른 데이터 전부 제거
movies=movies[['movie_id','title']]

In [10]:
"""
[ train set/ test set 으로 분리하기 ]
- 분리하지 않으면 정확도의 왜곡 생김
"""

'\n[ train set/ test set 으로 분리하기 ]\n- 분리하지 않으면 정확도의 왜곡 생김\n'

In [11]:
from sklearn.model_selection import train_test_split # sklearn_selection library 이용해 데이터 분리
x=ratings.copy() # ratings data 복사
y=ratings['user_id'] # user_id 기준으로 train set, test set 분리
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, stratify=y) # 75%-> train set(x_train), 25% -> test set(x_test), y_train & y_test -> user id저장   

In [12]:
# 정확도 계산하는 Function
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [13]:
"""
Score: 예측 model을 넘겨받아서 해당 모델로 예측한 예측치의 정확도(RMSE) 계산
"""

'\nScore: 예측 model을 넘겨받아서 해당 모델로 예측한 예측치의 정확도(RMSE) 계산\n'

In [14]:
import numpy as np
# Model 별 RMSE 계산하는 Function 
def score(model):
  id_pairs=zip(x_test['user_id'],x_test['movie_id'])  # test set의 user_id, movie_id를 pair로 짝맞춰 data만듬
  y_pred=np.array([model(user,movie) for (user,movie) in id_pairs]) # 예측모델에 의해 (user,movie)에 대한 예측값 계산해 list로 저장
  y_true=np.array(x_test['rating']) # 실제 평점값 list를 x_true에서 받아오기
  return RMSE(y_true, y_pred) # 예측값 list & 실제값 list에 대한 RMSE구하기

In [15]:
# train data로 Full Matrix구하기 -> 추천 알고리즘에 사용
rating_matrix=x_train.pivot(index='user_id',columns='movie_id',values='rating') 
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1630,1631,1632,1633,1635,1636,1637,1638,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1652,1653,1655,1656,1658,1659,1660,1661,1662,1663,1664,1666,1667,1668,1669,1671,1672,1673,1675,1679,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,NaN,5.0,4.0,1.0,5.0,3.0,2.0,NaN,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,NaN,4.0,3.0,2.0,NaN,NaN,3.0,3.0,5.0,NaN,NaN,NaN,2.0,2.0,3.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
"""
사용자 특성에 따라 집단 나눠서 예측하려면 평점 데이터 (u.data) + 사용자 데이터(u.user) 필요
=> 2개의 data를 merge => 평점 data의 full matrix구하기

* 모든 사용자가 영화에 대해서 평가하지는 않았으므로 NaN 존재
"""

'\n사용자 특성에 따라 집단 나눠서 예측하려면 평점 데이터 (u.data) + 사용자 데이터(u.user) 필요\n=> 2개의 data를 merge => 평점 data의 full matrix구하기\n\n* 모든 사용자가 영화에 대해서 평가하지는 않았으므로 NaN 존재\n'

In [17]:
# train set의 모든 사용자 pair의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy=rating_matrix.copy().fillna(0) # rating_matrix의 값을 복사하고 nan값을 0으로 채우기(안하면 에러발생)
user_similarity=cosine_similarity(matrix_dummy,matrix_dummy) # 모든 사용자 간의 코사인 유사도 계산 (계산 시 943차원 가짐)
user_similarity=pd.DataFrame(user_similarity,index=rating_matrix.index,columns=rating_matrix.index) # user_similarity에 인덱스 지정

In [18]:
"""
주어진 영화(movie_id)의 가중평균 rating을 계산
가중치는 주어진 사용자와 다른 사용자 간의 유사도 의미
"""

'\n주어진 영화(movie_id)의 가중평균 rating을 계산\n가중치는 주어진 사용자와 다른 사용자 간의 유사도 의미\n'

In [19]:
# user_id,movie_id에 대한 예상 평점을 CF알고리즘으로 계산 
def CF_simple(user_id,movie_id):
  if movie_id in rating_matrix:
    sim_scores=user_similarity[user_id].copy() # 사용자와 다른 사용자의 유사도 복사
    movie_ratings=rating_matrix[movie_id].copy() # 주어진 영화에 대한 모든 사용자의 평점 복사
    none_rating_idx=movie_ratings[movie_ratings.isnull()].index # 주어진 영화에 대해 평가하지 않은 사용자의 위치 파악 ( 가중평균에서 빼줌 )
    movie_ratings=movie_ratings.dropna() # 주어진 영화에 대해 평가하지 않은 사용자의 평점 제거 
    sim_scores=sim_scores.drop(none_rating_idx) # 평가하지 않은 사용자와의 유사도 제거
    mean_rating=np.dot(sim_scores,movie_ratings)/sim_scores.sum() # 주어진 영화를 평가한 각 사용자에 대해 평점을 유사도로 가중평균한 예측치 구하기
  else:
    mean_rating=3.0
  return mean_rating

score(CF_simple)

1.0204353962939046

In [20]:
"""
RMSE=1.016으로 이전의 방법들보다 개선
=> 유사도가 높은 사용자에게 가중치를 더 주어서 평균을 계산하면 더 정확한 예측치 구할 수 있음
"""

'\nRMSE=1.016으로 이전의 방법들보다 개선\n=> 유사도가 높은 사용자에게 가중치를 더 주어서 평균을 계산하면 더 정확한 예측치 구할 수 있음\n'

## 단순한 CF 알고리즘 개선하기 ( 이웃을 고려한 CF )
: 사용자 중 유사도 높은 사용자를 선정해 그 사람들의 평점만 가지고 예측 

### 이웃을 정하는 기준 
1. KNN (K Nearest Neighbors)
- 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명을 선택 
2. Thresholding
- 유사도의 기준(ex. 상관계수>0.8) 을 충족시키는 사용자를 이웃으로 정함
- KNN보다 정확하지만 정해진 기준 넘는 사용자가 많지 않음 


In [21]:
# 모델별 RMSE계산하는 함수 수정 
def modi_score(model, neighbor_size=0):
  id_pairs=zip(x_test['user_id'],x_test['movie_id'])  # test set의 user_id, movie_id를 pair로 짝맞춰 data만듬
  y_pred=np.array([model(user,movie,neighbor_size) for (user,movie) in id_pairs]) # 예측모델에 의해 (user,movie)에 대한 예측값 계산해 list로 저장
  y_true=np.array(x_test['rating']) # 실제 평점값 list를 x_true에서 받아오기
  return RMSE(y_true, y_pred) # 예측값 list & 실제값 list에 대한 RMSE구하기

In [22]:
# Neighbot size를 정해서 예측치 계산
def cf_knn(user_id,movie_id,neighbor_size=0):
  if movie_id in rating_matrix:
    sim_scores=user_similarity[user_id].copy()
    movie_ratings= rating_matrix[movie_id].copy()
    none_rating_idx=movie_ratings[movie_ratings.isnull()].index 
    movie_ratings=movie_ratings.drop(none_rating_idx)
    sim_scores=sim_scores.drop(none_rating_idx)

    # neighbor size가 지정되지 않는 경우
    if neighbor_size==0:
      mean_rating=np.dot(sim_scores,movie_ratings)/ sim_scores.sum()
    # neighbor size가 지정된 경우
    else: 
      if len(sim_scores)>1: # 영화를 평가한 사용자가 매우 작으면 계산에 문제가 생기기 때문 
        neighbor_size=min(neighbor_size,len(sim_scores)) # neighbor_size와 해당 영화 평가한 사용자 수 중 작은 걸 neighbor_size로 정하기 ( 사용자 수가 neighbor_size보다 작은 경우 고려)
        sim_scores=np.array(sim_scores) # argsort()사용하기 위해 dataframe을 arr로 바꿔줌 
        movie_ratings=np.array(movie_ratings) 
        user_idx=np.argsort(sim_scores) # 다른 사용자를 유사도 순으로 정렬
        sim_scores=sim_scores[user_idx][-neighbor_size:] # 유사도가 높은 neighbor_size 명의 사용자 선정
        movie_ratings=movie_ratings[user_idx][-neighbor_size:] # 선정된 사용자의 주어진 영화에 대한 평점 추출
        mean_rating=np.dot(sim_scores,movie_ratings)/sim_scores.sum() # 선정된 이웃 사용자의 해당 영화에 대한 평점을 유사도로 가중한 평점평균 구하기 
      else: 
        mean_rating=3.0
        
  else:
    mean_rating=3.0
  return mean_rating

modi_score(cf_knn,neighbor_size=30)


1.0117820871365162

 ### 결과 
 : 이웃의 크기를 고려하지 않은 simple_cf 보다 개선 

## 실제 추천 받는 기능 구현 
- 추천 받을 ( 사용자id, 추천 아이템 수, 이웃의 크기 ) 를 매개변수로 하고 추천 영화 리스트를 return하는 Function

In [23]:
"""
실제 영화 추천 시 train/test set이 필요하지 X
=> 전체 data이용해서 full matrix와 사용자 간의 코사인 유사도(user_similarity)를 구해줌
"""
rating_matrix=ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy=rating_matrix.copy().fillna(0)
user_similarity=cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity=pd.DataFrame(user_similarity,index=rating_matrix.index,columns=rating_matrix.index)

In [24]:
def recom_movie(user_id,n_items,neighbor_size=30):
  user_movie=rating_matrix.loc[user_id].copy() # 전체 data(rating_matrix) 중 user_id의 평점 데이터 복사 
  for movie in rating_matrix: # 전체 data에 대해 탐색
    if pd.notnull(user_movie.loc[movie]): # 현 영화 평점이 0이 아닌경우 => 이미 본 영화임 
      user_movie.loc[movie]=0 # 추천list에서 제거하기 위해 0으로 초기화 
    else: # 아직 보지 않은 영화 
      user_movie.loc[movie]=cf_knn(user_id,movie,neighbor_size) #knn이용해 user_id의 예상 평점 계산 
  movie_sort=user_movie.sort_values(ascending=False)[:n_items] # 예상평점(user_movie) 기준으로 영화 리스트을 내림차순으로 정렬 => 상위 n_items개 저장 
  recom_movies=movies.loc[movie_sort.index] # 영화 정보(movies)에서 골라진 영화의 정보 추출
  recomendations=recom_movies['title'] # 추출된 정보(recom_movies)에서 'title'추출
  return recomendations 
  
recom_movie(user_id=2,n_items=5,neighbor_size=30)

movie_id
1293                     Ayn Rand: A Sense of Life (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

## 최적의 이웃 크기 설정